In [15]:
import pandas as pd
from pandas import DataFrame as df
import numpy as np
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder,PolynomialFeatures
from statsmodels.distributions.empirical_distribution import ECDF
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score,precision_score, recall_score,confusion_matrix,classification_report

# Rule based
애초에 grid 2와 18은 제거됨

In [4]:
total_data=pd.read_csv('./total_data.csv')
total_data=total_data.loc[:,['x','y','단속일시','grid_num']]
total_data['단속일시']=total_data['단속일시'].astype('datetime64[ns]')
total_data.head()

,x,y,단속일시,grid_num
0,37.551987,127.007976,2015-01-01 00:13:00,2
1,37.551987,127.007976,2015-01-01 00:14:00,2
2,37.551987,127.007976,2015-01-01 00:15:00,2
3,37.551987,127.007976,2015-03-16 21:08:00,2
4,37.551987,127.007976,2015-03-16 21:08:00,2


In [5]:
# 시간대 별로 나누기
def time_zone(data_i):
    if data_i>=6 and data_i<12:
        return 0
    if data_i>=12 and data_i<18:
        return 1
    if data_i>=18 and data_i<23:
        return 2
    else:
        return 3

In [6]:
# Holiday 가져오기
import requests
holiday=[]
for year in range(2015,2021):
    for month in ['01','02','03','04','05','06','07','08','09','10','11','12']:
        URL = "http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getHoliDeInfo?ServiceKey=SHKN0mFGJKABDNHU9MEyR3vq4%2BE5jaHvVaP8eUt2JgOX1tIWGtc3JvTbwr4a2MRDYa6ONNE0tb7ADSabcrBT7w%3D%3D&_type=json&solYear={year}&solMonth={month}".format(year=year,month=month)
        html = requests.get(URL)
        if html.json()['response']['body']['items']:
            try:
                holiday.append(html.json()['response']['body']['items']['item']['locdate'])
            except:
                for i in html.json()['response']['body']['items']['item']:
                    holiday.append(i['locdate'])             
# int 2 str
holiday=[str(i) for i in holiday]

In [7]:
# 하루를 24등분해서 2015년 1월 1일부터 2020년 7월 14일까지의 데이터를 작성 
# 일단 그리드만을 대상으로 작성
processed_data=[]
for j in set(total_data.grid_num):
    print(j)
    x=total_data.loc[total_data['grid_num']==j,:]
    new_data=df(index=pd.date_range('2015-01-01','2020-07-14',freq='H'))
    new_data['단속건수']=0
    for i in new_data.index:
        new_data['단속건수'][i]=((i<=x.단속일시) & (x.단속일시<i+dt.timedelta(hours=1))).sum()
    # date란 feature을 만들어주자
    new_data['date']=new_data.index
    new_data['연월시']=new_data['date'].apply(lambda i : '%d'%(i.year)+('%d'%(i.month)).zfill(2)+('%d'%(i.day)).zfill(2))
    new_data['month']=new_data['date'].apply(lambda i : i.month)
    new_data['days']=new_data['date'].apply(lambda i : i.day)
    new_data['hour']=new_data['date'].apply(lambda i : i.hour)
    new_data['grid_num']=j-1
    new_data['요일']=new_data['date'].apply(lambda i : dt.datetime.weekday(i)) # 0 : 월 ~ 6 : 일
    new_data['공휴일']=((new_data['요일']==6)|(new_data['연월시'].apply(lambda i : True if i in holiday else False))).astype(int)
    processed_data.append(new_data.values.tolist())

1
2
4


KeyboardInterrupt: 

In [ ]:
processed_data=df(np.array(processed_data).reshape(-1,10),columns=['단속건수','date','연월시','month','days','hour','grid_num','요일','시간대','공휴일'])
processed_data.head()

# 데이터 현황
전체 데이터 개수 : 1,115,615
단속건수가 0이 아닌 데이터 개수 : 221,302   

In [95]:
for i in set(processed_data.grid_num):
    print('grid_num : %d'%i)
    print('data 개수 : %d'%len(processed_data.loc[processed_data.grid_num==i][:]))
    print('단속건수가 0이 아닌 data 개수 : %d'%(processed_data.loc[processed_data.grid_num==i]['단속건수']!=0).sum())

grid_num : 0
data 개수 : 48505
단속건수가 0이 아닌 data 개수 : 9969
grid_num : 1
data 개수 : 48505
단속건수가 0이 아닌 data 개수 : 8437
grid_num : 3
data 개수 : 48505
단속건수가 0이 아닌 data 개수 : 16876
grid_num : 4
data 개수 : 48505
단속건수가 0이 아닌 data 개수 : 16657
grid_num : 5
data 개수 : 48505
단속건수가 0이 아닌 data 개수 : 2
grid_num : 6
data 개수 : 48505
단속건수가 0이 아닌 data 개수 : 12103
grid_num : 7
data 개수 : 48505
단속건수가 0이 아닌 data 개수 : 21145
grid_num : 8
data 개수 : 48505
단속건수가 0이 아닌 data 개수 : 18
grid_num : 9
data 개수 : 48505
단속건수가 0이 아닌 data 개수 : 23417
grid_num : 10
data 개수 : 48505
단속건수가 0이 아닌 data 개수 : 10945
grid_num : 11
data 개수 : 48505
단속건수가 0이 아닌 data 개수 : 843
grid_num : 12
data 개수 : 48505
단속건수가 0이 아닌 data 개수 : 1922
grid_num : 13
data 개수 : 48505
단속건수가 0이 아닌 data 개수 : 17716
grid_num : 14
data 개수 : 48505
단속건수가 0이 아닌 data 개수 : 1
grid_num : 15
data 개수 : 48505
단속건수가 0이 아닌 data 개수 : 12426
grid_num : 16
data 개수 : 48505
단속건수가 0이 아닌 data 개수 : 6391
grid_num : 17
data 개수 : 48505
단속건수가 0이 아닌 data 개수 : 4
grid_num : 19
data 개수 : 48505
단속건수가 0이 아닌 da

In [97]:
# 데이터 저장
processed_data.to_csv('./processed_data_중구.csv',index=False)

In [9]:
# 데이터 로드
processed_data=pd.read_csv('./processed_data_중구.csv')

In [10]:
processed_data=processed_data.loc[:,['단속건수','date','연월시','month','days','hour','grid_num','요일','공휴일']]

In [11]:
processed_data

,단속건수,date,연월시,month,days,hour,grid_num,요일,공휴일
0,0,2015-01-01 00:00:00,20150101,1,1,0,0,3,1
1,0,2015-01-01 01:00:00,20150101,1,1,1,0,3,1
2,0,2015-01-01 02:00:00,20150101,1,1,2,0,3,1
3,0,2015-01-01 03:00:00,20150101,1,1,3,0,3,1
4,0,2015-01-01 04:00:00,20150101,1,1,4,0,3,1
...,...,...,...,...,...,...,...,...,...
1115610,0,2020-07-13 20:00:00,20200713,7,13,20,24,0,0
1115611,0,2020-07-13 21:00:00,20200713,7,13,21,24,0,0
1115612,0,2020-07-13 22:00:00,20200713,7,13,22,24,0,0
1115613,0,2020-07-13 23:00:00,20200713,7,13,23,24,0,0


# model 1
단속 건수가 0인 경우 0, 아닌 경우는 1로 바꿔서 분류 진행
여기에서 단속 건수가 1이 될 확률이 70%인 경우 model2, 즉 몇 건으로 단속될 것 인 지를 예측

In [13]:
X=processed_data.loc[:,['month','days','hour','grid_num','요일','공휴일']]

# 1차항 만 고려했을 경우

In [18]:
X=processed_data.loc[:,['month','days','hour','grid_num','요일','공휴일']]
one=OneHotEncoder()
one.fit(X)
X_one=one.transform(X) 
y=(processed_data.loc[:,'단속건수']!=0).astype(int)

## Train Test Split

In [19]:
X_train,X_test,y_train,y_test=train_test_split(X_one,y,test_size=0.1)

# xgboost 학습

In [20]:
xgb=XGBClassifier()
xgb.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

### TEST - 1이 될 확률이 30%이상인 경우 1로 분류

In [22]:
p = xgb.predict_proba(X_test)
predicted=p[:,1]>0.3 
print(confusion_matrix(y_test,predicted))
print(classification_report(y_test,predicted))

[[76433 12816]
 [ 4963 17350]]
              precision    recall  f1-score   support

           0       0.94      0.86      0.90     89249
           1       0.58      0.78      0.66     22313

    accuracy                           0.84    111562
   macro avg       0.76      0.82      0.78    111562
weighted avg       0.87      0.84      0.85    111562



# 2차항(총 feature 28)

In [23]:
poly=PolynomialFeatures(degree=2)
X_=poly.fit_transform(X)
one=OneHotEncoder()
one.fit(X_)
X_one=one.transform(X_) 
y=(processed_data.loc[:,'단속건수']!=0).astype(int)

## Train, test split

In [24]:
X_train,X_test,y_train,y_test=train_test_split(X_one,y,test_size=0.1)

# xgboost 학습

In [25]:
xgb=XGBClassifier()
xgb.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [189]:
xgb.save_model('model1_1.model')

### TEST - 1이 될 확률이 20%이상인 경우 1로 분류

In [26]:
p = xgb.predict_proba(X_test)
predicted=p[:,1]>0.2 # 1로 분류할 확률이 20%이상인 경우를 1로 분류
print(confusion_matrix(y_test,predicted))
print(classification_report(y_test,predicted))

[[69567 20213]
 [ 2497 19285]]
              precision    recall  f1-score   support

           0       0.97      0.77      0.86     89780
           1       0.49      0.89      0.63     21782

    accuracy                           0.80    111562
   macro avg       0.73      0.83      0.74    111562
weighted avg       0.87      0.80      0.81    111562



### TEST - 1이 될 확률이 30%이상인 경우 1로 분류

In [27]:
p = xgb.predict_proba(X_test)
predicted=p[:,1]>0.3 # 1로 분류할 확률이 30%이상인 경우를 1로 분류
print(confusion_matrix(y_test,predicted))
print(classification_report(y_test,predicted))

[[76817 12963]
 [ 4730 17052]]
              precision    recall  f1-score   support

           0       0.94      0.86      0.90     89780
           1       0.57      0.78      0.66     21782

    accuracy                           0.84    111562
   macro avg       0.76      0.82      0.78    111562
weighted avg       0.87      0.84      0.85    111562



### TEST - 1이 될 확률이 40%이상인 경우 1로 분류

In [28]:
#############################TEST#############################################
p = xgb.predict_proba(X_test)
predicted=p[:,1]>0.4 # 1로 분류할 확률이 40%이상인 경우를 1로 분류
print(confusion_matrix(y_test,predicted))
print(classification_report(y_test,predicted))

[[81058  8722]
 [ 6884 14898]]
              precision    recall  f1-score   support

           0       0.92      0.90      0.91     89780
           1       0.63      0.68      0.66     21782

    accuracy                           0.86    111562
   macro avg       0.78      0.79      0.78    111562
weighted avg       0.86      0.86      0.86    111562



### TEST - 1이 될 확률이 50%이상인 경우 1로 분류

In [29]:
p = xgb.predict_proba(X_test)
predicted=p[:,1]>0.5 # 1로 분류할 확률이 50%이상인 경우를 1로 분류
print(confusion_matrix(y_test,predicted))
print(classification_report(y_test,predicted))

[[84370  5410]
 [ 9754 12028]]
              precision    recall  f1-score   support

           0       0.90      0.94      0.92     89780
           1       0.69      0.55      0.61     21782

    accuracy                           0.86    111562
   macro avg       0.79      0.75      0.77    111562
weighted avg       0.86      0.86      0.86    111562

